## Part 1: Pulse Rate Algorithm

### Contents
Fill out this notebook as part of your final project submission.

**You will have to complete both the Code and Project Write-up sections.**
- The [Code](#Code) is where you will write a **pulse rate algorithm** and already includes the starter code.
   - Imports - These are the imports needed for Part 1 of the final project. 
     - [glob](https://docs.python.org/3/library/glob.html)
     - [numpy](https://numpy.org/)
     - [scipy](https://www.scipy.org/)
- The [Project Write-up](#Project-Write-up) to describe why you wrote the algorithm for the specific case.


### Dataset
You will be using the **Troika**[1] dataset to build your algorithm. Find the dataset under `datasets/troika/training_data`. The `README` in that folder will tell you how to interpret the data. The starter code contains a function to help load these files.

1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----

### Code

In [1]:
import numpy as np
import glob

import scipy as sp
import scipy.io
import scipy.signal

def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]

def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)

def BandpassFilter(signal, pass_band, fs):
    """Bandpass Filter.
    
    Args:
        signal: (np.array) The input signal
        pass_band: (tuple) The pass band. Frequency components outside 
            the two elements in the tuple will be removed.
        fs: (number) The sampling rate of <signal>
        
    Returns:
        (np.array) The filtered signal
    """
    b, a = sp.signal.butter(3, pass_band, btype='bandpass', fs=fs)
    return sp.signal.filtfilt(b, a, signal)

def PreprocessSignal(ppg, accx, accy, accz):
    """Preprocess Signal.
    
    Apply Bandpass filter with 40-240BPM range to raw signals.
    
    Args:
        numpy arrays for raw ppg, accx, accy, accz signals.
        
    Returns:
        numpy arrays for filtered ppg, accx, accy, accz signals.
    """    
    fs=125
    freq_filter=(40/60,240/60)
    ppg=BandpassFilter(ppg, freq_filter, fs)
    accx=BandpassFilter(accx, freq_filter, fs)
    accy=BandpassFilter(accy, freq_filter, fs)
    accz=BandpassFilter(accz, freq_filter, fs)
    return ppg, accx, accy, accz

def RunPulseRateAlgorithm(data_fl, ref_fl):
    """Estimate pulse rate from PPG and accelerometer signals. 
    
    Calculate MAE errors and confidence.
        
    Args:
        Paths for PPG and accelerometer signal files and pulse rate reference.
        
    Returns:
        Return per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays.
    """    
    # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    ppg, accx, accy, accz = PreprocessSignal(ppg, accx, accy, accz)
    
    # Compute pulse rate estimates and estimation confidence.
    window_len=600
    fs=125
    # Generate outputs every 2s
    window_shift=fs*2
    
    # pr:estimated pulse rate
    pr=[]
    confidence=[]
    
    for i in range(0, len(ppg) - window_len, window_shift):
        ppg_seg=ppg[i: i + window_len]
        accx_seg=accx[i: i + window_len]
        accy_seg=accy[i: i + window_len]
        accz_seg=accz[i: i + window_len]
        
        # Run fast Fourier transform on each window
        freqs = np.fft.rfftfreq(window_len, 1/fs)
        fft_p = abs(np.fft.rfft(ppg_seg, 600))
        fft_x = abs(np.fft.rfft(accx_seg, 600))
        fft_y = abs(np.fft.rfft(accy_seg, 600))
        fft_z = abs(np.fft.rfft(accz_seg, 600))
        
        fft_acc=np.sqrt(np.square(fft_x)+np.square(fft_y)+np.square(fft_z))
        
        # Find peaks within normal pulse frequency from PPG and accelerometer signals after fft
        fftp_pks_raw,fftp_h_raw = sp.signal.find_peaks(fft_p,height=250)
        fftp_pks_raw=freqs[fftp_pks_raw]
        fftp_h_raw=fftp_h_raw['peak_heights']

        fftp_f_range=(fftp_pks_raw>=40/60)&(fftp_pks_raw<=240/60)
        fftp_pks=fftp_pks_raw[fftp_f_range]
        fftp_h=fftp_h_raw[fftp_f_range]    

        fftacc_pks_raw,fftacc_h_raw = sp.signal.find_peaks(fft_acc,height=2)
        fftacc_pks_raw=freqs[fftacc_pks_raw]
        fftacc_h_raw=fftacc_h_raw['peak_heights']

        fftacc_f_range=(fftacc_pks_raw>=40/60)&(fftacc_pks_raw<=240/60)
        fftacc_pks=fftacc_pks_raw[fftacc_f_range]
        fftacc_h=fftacc_h_raw[fftacc_f_range]

        
        # If PPG frequencies peaks were found, compare the top-4 dominant peaks with the top-4 dominant peaks 
        # in accelerometer signals and remove shared peaks to reduce movement artifacts.
        if len(fftp_pks)>0:
            fftp_pks=fftp_pks[np.argsort(-fftp_h)]
            fftp_h=fftp_h[np.argsort(-fftp_h)]    
            if len(fftp_pks)>4:
                fftp_pks=fftp_pks[:4]
                fftp_h=fftp_h[:4]

            if len(fftacc_pks)>0:
                fftacc_pks=fftacc_pks[np.argsort(-fftacc_h)]
                fftacc_h=fftacc_h[np.argsort(-fftacc_h)]
            if len(fftacc_pks)>4:
                fftacc_pks=fftacc_pks[:4]
                fftacc_h=fftacc_h[:4]

            unique_f=[x for x in fftp_pks if not x in fftacc_pks]
            if len(unique_f)>0:
                pr_f=unique_f[0]
            else:
                pr_f=fftp_pks[0]
        
        # If no peak was found, use the strongest PPG frequency as the estimate.
        else:
            freqs_range=(freqs>=40/60)&(freqs<=240/60)
            freqs_filtered=freqs[freqs_range]
            fft_p_filtered=fft_p[freqs_range]

            pr_f=freqs_filtered[np.argsort(-fft_p_filtered)][0]
        
        # Pulse rate usually doesn't change significantly within 2s, I limit the change to 1.1 times to smooth the signal.
        pr_r=pr_f*60
        if len(pr)>0:
            if pr_r>1.1*pr[-1]:
                pr_r=1.1*pr[-1]
            if pr_r<pr[-1]/1.1:
                pr_r=pr[-1]/1.1

        pr.append(pr_r)
        
        # Find the nearest peak of the estimated pulse frequency and calculate the confidence.
        delta_fftp_pks_raw=abs(fftp_pks_raw-pr_r/60)
        confidence.append(fftp_h_raw[delta_fftp_pks_raw.argmin()]/sum(fftp_h_raw))

    
    # Return per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays.
    reference=sp.io.loadmat(ref_fl)['BPM0'].reshape((1,-1))[0]
    errors=abs(pr[:len(reference)]-reference)
    confidence=np.array(confidence[:len(reference)])
    
    return errors, confidence

In [2]:
Evaluate()

11.127948293870112

-----
### Project Write-up

Answer the following prompts to demonstrate understanding of the algorithm you wrote for this specific context.

> - **Code Description** - Include details so someone unfamiliar with your project will know how to run your code and use your algorithm. 
> - **Data Description** - Describe the dataset that was used to train and test the algorithm. Include its short-comings and what data would be required to build a more complete dataset.
> - **Algorithhm Description** will include the following:
>   - how the algorithm works
>   - the specific aspects of the physiology that it takes advantage of
>   - a describtion of the algorithm outputs
>   - caveats on algorithm outputs 
>   - common failure modes
> - **Algorithm Performance** - Detail how performance was computed (eg. using cross-validation or train-test split) and what metrics were optimized for. Include error metrics that would be relevant to users of your algorithm. Caveat your performance numbers by acknowledging how generalizable they may or may not be on different datasets.

Your write-up goes here...

- **Code Description** 

The code can be used to estimate pulse rates according to PPG and accelerometer signals. The performance of the estimation is then evaluated. Each function is described by the docstrings. "Evaluate" function should be ran to initiate the whole process.


- **Data Description** 

Troika dataset was used to validate and test the algorithm. PPG signals, three-axis acceleration signals, and one-channel ECG signals were simultaneously recorded from 12 subjects with age from 18 to 35. Each recording is 5 min during rest and different running speeds. The ground-truth of heart rate is calculated from the ECG signal.

The recordings are relatively short. Longer recordings and recordings during other activities are needed to build a more complete dataset. More information of the subjects will also be helpful.


- **Algorithm Description** 

The amount of blood within wrist capillaries is affected by ventricle contractions. PPG sensor measures the light absorption by red blood cells as an indication of the amount of blood to estimate the pulse rate.

Arm motion can affect PPG sensor measurements. To reduce motion-caused noise, arm acceleration is monitored at the same time. Fast Fourier transform are performed in both PPG and accelerometer signals in 4.8s window every 2s. The top-4 strongest frequency peaks within normal pulse rate range are compared between the two channels. The strongest peak appears in PPG signal but no in accelerometer signal is used as the estimates. If all peaks overlap, the strongest peak in PPG signal is used. The pulse rate estimate of each time point is also restricted to 1.1 times of the previous estimate.

The algorithm generates pulse rate estimates and confidence every 2s. Confidence is measured as the percentage of energy in the frequency spectrum near the pulse frequency estimate. The energy near the pulse frequency estimate is an indication of signal strength. The energy elsewhere is an indication of noise strength. Their ratio can be a measurement for the confidence of estimates.

The algorithm may not be accurate if pulse rate changes rapidly in short time.

The algorithm will likely fail when arm acceleration is very strong and irregular. Finger movements may not be captured by accelerometer, which also affects the accuracy of the algorithm.
  
  
- **Algorithm Performance** 

Mean absolute error of the best 90% of the estimates is used to evaluate algorithm performance. The algorithm can be used in other datasets with PPG and three-axis acceleration signals during rest and running.


-----
### Next Steps
You will now go to **Test Your Algorithm** to apply a unit test to confirm that your algorithm met the success criteria. 